In [8]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn

In [9]:
df = pd.read_csv('Resources/tweet_data.csv')

In [24]:
X = df['tidy_tweet'].astype(str)
ylabels = df['target']/4

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

In [15]:
X_val = X_train[:10000]
y_val = y_train[:10000]

In [22]:
run_name = "Twitter_Sentiment"
with mlflow.start_run(run_name=run_name) as run:
    model = "https://tfhub.dev/google/universal-sentence-encoder/4"
    hub_layer = hub.KerasLayer(model, output_shape=[512], input_shape=[], 
                           dtype=tf.string, trainable=True)
    
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=20000,
                    validation_data=(X_val, y_val),
                    verbose=1)

    #Save so that we can easily load this in MLflow later if we need to
    mlflow.sklearn.save_model(model, "saved_model", 
                              serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_CLOUDPICKLE)
    mlflow.log_artifact("saved_model")

Epoch 1/20
42/42 [==============================] - 212s 5s/step - loss: -0.1046 - accuracy: 0.1693 - val_loss: -1.1483 - val_accuracy: 0.4347
Epoch 2/20
42/42 [==============================] - 211s 5s/step - loss: -2.5641 - accuracy: 0.5051 - val_loss: -4.2919 - val_accuracy: 0.5820
Epoch 3/20
42/42 [==============================] - 218s 5s/step - loss: -6.4151 - accuracy: 0.5930 - val_loss: -9.0028 - val_accuracy: 0.6209
Epoch 4/20
42/42 [==============================] - 223s 5s/step - loss: -11.9446 - accuracy: 0.6532 - val_loss: -15.0999 - val_accuracy: 0.6846
Epoch 5/20
42/42 [==============================] - 219s 5s/step - loss: -18.6986 - accuracy: 0.6867 - val_loss: -22.1936 - val_accuracy: 0.7075
Epoch 6/20
42/42 [==============================] - 202s 5s/step - loss: -26.4066 - accuracy: 0.7022 - val_loss: -30.1730 - val_accuracy: 0.7126
Epoch 7/20
42/42 [==============================] - 203s 5s/step - loss: -35.0973 - accuracy: 0.7109 - val_loss: -38.9240 - val_accuracy

AttributeError: 'Sequential' object has no attribute 'score'

In [ ]:
!cat saved_model/conda.yaml

TypeError: can't pickle _thread.RLock objects